# Парсинг и трансформация данных

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import re
import random
import json

import requests
from bs4 import BeautifulSoup
import base64
import lxml

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import pandas as pd

from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy import distance
from geopy.extra.rate_limiter import RateLimiter

pd.set_option('max_colwidth', 600)
pd.set_option('display.width', 600)

In [3]:
# Используем url c текстовой таблицей
URL = "http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&v_id=1&pN="

data = []
for page in range(5):
    print(f"Working with {page+1} page")
    full_url = URL + str(page+1)
    response = requests.get(full_url)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    table = soup.find("table", class_="tbrd")
    rows = table.find_all("tr")

    # делим по разметке на тексты
    delimiter = '</td></tr>\n<tr class=\"tbb\">'
    result = str(rows[0]).split(delimiter) 

    data.extend(result)
    print(f"Page #{page+1} sucsefully added")


Working with 1 page
Page #1 sucsefully added
Working with 2 page
Page #2 sucsefully added
Working with 3 page
Page #3 sucsefully added
Working with 4 page
Page #4 sucsefully added
Working with 5 page
Page #5 sucsefully added


In [4]:
data_clean = [t.split('\">')[1] for t in data] # if t.strip()
data_clean = [t.split("</td>")[0] for t in data_clean]
data_clean = [t for t in data_clean if "<" not in t] # if t.strip()

In [5]:
data_clean[:3]

['27.08.2023.  Продам двухкомнатную квартиру.  ул. Советская, дом 174, этаж 5/9. Общая площадь - 49.80 кв.м., жилая площадь - 30.00 кв.м., кухня - 9.00 кв.м.. , id:26097. \r\nВНИМАНИЕ! ЭКСКЛЮЗИВНОЕ ПРЕДЛОЖЕНИЕ РИО-ЛЮКС!\r\nПредлагается к покупке ДВУХКОМНАТНАЯ квартира в идеальном состоянии!\r\n\r\nИнфраструктура\r\nОтличный экологически благополучный район, есть все для комфортной жизни.\r\nВ шаговой доступности:\r\n\r\nУдобная транспортная развязка в любой район города\r\nТорговый центр «Тройка», магазины «Монетка», «Магнит», «Пятерочка», «Красно-белое».\r\nАптеки, парикмахерские, кафе, кондитерские.\r\n\r\nПространство для детей\r\n\r\n\r\nСовременные детские и спортивные площадки.\r\n\r\n\r\n\r\nДля малышей в шаговой доступности, детские сады  17,  30,  136,  49,  9,\r\n\r\n\r\n\r\nДля детей постарше школа 10,  32,  50,  1\r\nКрытый каток Умка, спортивные секции, ФОК «Дюна»\r\n\r\nО доме\r\n\r\n\r\nДом панельный, 9-ти этажный, качество постройки по ГОСТ!!!\r\nОтличная шумоизоляция, 

In [6]:
df = pd.DataFrame(data_clean, columns=['text'])
df = df.drop_duplicates().reset_index(drop=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    465 non-null    object
dtypes: object(1)
memory usage: 3.8+ KB


## transform

In [8]:
df['date'] = df['text'].apply(lambda x: x.split('.  Продам')[0])
df['note_len'] = df['text'].apply(lambda x: len(x))
df = df.loc[df['note_len'] > 3]

df['date'] = pd.to_datetime(df['date'], format="%d.%m.%Y")

df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['dayofweek'] = df['date'].dt.dayofweek #Monday=0, Sunday=6

In [9]:
df['rooms'] = df['text'].apply(lambda x: x.split("Продам ")[1].split('квартиру')[0])

In [10]:
def district_sort(text):
    text = text.lower()
    if 'орджоникидз' in text:
        if 'левый берег' in text or 'левом' in text:
            return 'Орджоникидзевский(левый берег)'
        else:
            return 'Орджоникидзевский'
    elif 'правобереж' in text:
        return 'Правобережный'
    elif 'ленинск' in text:
        if 'левый берег' in text or 'левом' in text:
            return 'Ленинский(левый берег)'
        else:
            return 'Ленинский'

In [11]:
df['district'] = df['text'].apply(district_sort)
df['text'] = df['text'].map(lambda x: x.replace('ул. ул. ', 'ул. '))
df['street'] = df['text'].apply(lambda x: x.split("ул.")[1].split(', этаж')[0] if "ул." in x else 'unknown')
df['street'] = df['street'].map(lambda x: x.replace('ул ', ''))

df['street'] = df['street'].apply(lambda x: x.lstrip(". ").rstrip(", ")).map(lambda x: x.replace('дом ', ''))

In [12]:
df[['floor', 'total_floors']] = df['text'].apply(
    lambda x: x.split("этаж")[1].split(',')[0].split('.')[0] if "этаж" in x else 'unknown').str.split('/', expand=True)

In [13]:
df['total_area'] = df['text'].apply(lambda x: x.split("Общая площадь - ")[1].split(' кв.м.')[0] if "Общая площадь" in x else 'unknown')
df['living_area'] = df['text'].apply(lambda x: x.split("жилая площадь - ")[1].split(' кв.м.')[0] if "жилая площадь - " in x else 'unknown')
df['kitchen_area'] = df['text'].apply(lambda x: x.split("кухня - ")[1].split(' кв.м.')[0] if "кухня - " in x else 'unknown')

df['price'] = df['text'].apply(lambda x: x.split("Цена - ")[1].split(' т.р.')[0] if "Цена - " in x else None)

In [14]:
df['street'] = 'Магнитогорск, ' + df['street']

In [15]:
geolocator = Nominatim(user_agent="my-app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df['location'] = df['street'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Магнитогорск, Ленина пр-т, 135',), **{}).
Traceback (most recent call last):
  File "c:\Users\user\Documents\GitHub\Magnitogorsk_real_estate\.venv\lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "c:\Users\user\Documents\GitHub\Magnitogorsk_real_estate\.venv\lib\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\user\AppData\Local\Programs\

In [16]:
df['lat_long'] = df['location'].dropna().apply(lambda x: x[1])

In [17]:
df.loc[df['district'].isna(), 'district'] = df['location'].dropna().apply(lambda x: x[0].split('район')[0].split(', ')[-1] if 'район' in str(x) else 'unknown')

parks_and_center = {'park_eko':(53.402233, 58.952641),
'attraction':(53.381266, 58.953624),
'south':(53.351887, 58.990688),
'eternal_fire':(53.405914, 58.991158),
'veter':(53.429143, 59.000213),
'city_center':(53.407164, 58.980285)}

In [18]:
# Итерирование по ключам словаря parks
for key in parks_and_center:
    # Создание нового столбца с именем ключа и заполнение его значениями
    df[key] = [(parks_and_center[key])]* len(df['lat_long'])
    df[key] = df[[key, 'lat_long']].dropna().apply(lambda row: round(distance.distance(row[key], row["lat_long"]).km, 2), axis=1)
df.info()
df['rooms'].value_counts()
df['rooms'][0]

<class 'pandas.core.frame.DataFrame'>
Index: 464 entries, 0 to 464
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   text          464 non-null    object        
 1   date          464 non-null    datetime64[ns]
 2   note_len      464 non-null    int64         
 3   day           464 non-null    int32         
 4   month         464 non-null    int32         
 5   year          464 non-null    int32         
 6   dayofweek     464 non-null    int32         
 7   rooms         464 non-null    object        
 8   district      393 non-null    object        
 9   street        464 non-null    object        
 10  floor         464 non-null    object        
 11  total_floors  464 non-null    object        
 12  total_area    464 non-null    object        
 13  living_area   464 non-null    object        
 14  kitchen_area  464 non-null    object        
 15  price         460 non-null    object        


'двухкомнатную '

In [19]:
def rooms_num(rooms):
    rooms = rooms.rstrip()
    answ = -1
    if rooms == 'однокомнатную':
        answ = 1
    if rooms == 'двухкомнатную':
        answ = 2
    if rooms == 'трехкомнатную':
        answ = 3
    if rooms == 'четырехкомнатную':
        answ = 4
    if rooms == 'многокомнатную':
        answ = 5
    return answ

In [20]:
df['rooms'] = df['rooms'].apply(rooms_num)
df['district'].value_counts()

district
Орджоникидзевский                 129
Ленинский                         103
Правобережный                      88
Орджоникидзевский                  24
Ленинский                          17
Правобережный                      15
Орджоникидзевский(левый берег)     14
unknown                             1
Ленинский(левый берег)              1
141-ый микро                        1
Name: count, dtype: int64

In [21]:
df[['floor', 'total_floors', 'total_area', 'living_area', 'kitchen_area', 'price']] = \
df[['floor', 'total_floors', 'total_area', 'living_area', 'kitchen_area', 'price']].apply(pd.to_numeric, errors='coerce').dropna()

df['district'] = df['district'].dropna().apply(lambda x: x.lstrip().rstrip())

df = df.drop(['text',  'note_len', 'location'], axis=1)

df['price'] = df['price'].dropna()

df = df.dropna(subset=['price'])

df['price_sq_meter'] = df['price'] / df['total_area']
df['floor_type'] = 'middle'

df.loc[df.floor == 1, 'floor_type'] = 'first'
df.loc[(df.floor == df.total_floors) & (df.floor != 1), 'floor_type'] = 'last'

In [22]:
df.to_csv('df_2.csv')